In [ ]:
import os
import requests
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharactorTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from weaviate import Client
from weaviate.embedded import EmbeddedOptions
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
OPENAI_API_KEY = "OPENAI_API_KEY"

In [ ]:
os.environ("OPENAI_API_KEY") = OPENAI_API_KEY

Data Ingestion

In [2]:
url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"

In [ ]:
response = requests.get(url)
response

In [ ]:
response.text

In [ ]:
raw_data = response.text

In [ ]:
with open("state_of_the_union.txt", "w") as f:
    f.write(raw_data)

In [ ]:
text_loader = TextLoader(r"state_of_the_union.txt")

In [ ]:
document = text_loader.load()
document[0].page_content  

Chunking the data

In [ ]:
text_splitter = RecursiveCharactorTextSplitter(chunk_size=500, chunk_over=50)

In [ ]:
text_chunks = text_splitter.split_documents(document)

In [ ]:
text_chunks[0].page_content

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
vector_store = FAISS.from_documents(text_chunks, embeddings)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
template = """

You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
output_parser = StrOutputParser()

Loading the model

In [ ]:
llm_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo")

In [ ]:
rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [ ]:
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")